# Battle of the Cities: Getting Venues from Foursquare API

In [1]:
import requests # library to handle requests
import pandas as pd 
import numpy as np
import json

### Bring in data

In [8]:
sf_hoods = pd.read_csv('../data/sf-locations.csv',delimiter=",")
nyc_hoods = pd.read_csv('../data/nyc-locations.csv')

sf_hoods.head()

Neighborhood  Latitude  Longitude
0     Anza Vista   37.7808  -122.4432
1   Alamo Square   37.7764  -122.4346
2  Sutro Heights   37.7782  -122.5083
3       Seacliff   37.7868  -122.4890
4    Lake Street   37.7856  -122.4794

## Foursquare requests

In [3]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IW1DNBRJOUHECGXR0ST002ZPDJYOBCIRTROZZ5YJZFHCIP3Q
CLIENT_SECRET:TWMIYINYFXOLI0IGNPXXLCIRHFWLUNJ4VP2TJZXSJGUP2ZBN


### Generate url for request

In [9]:
LIMIT = 50    
def getNearbyVenues(names, latitudes, longitudes, radius=300):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Turn `nearby_venues` into its own dataframe

In [10]:
sf_venues = getNearbyVenues(names=sf_hoods['Neighborhood'],
                                   latitudes=sf_hoods['Latitude'],
                                   longitudes=sf_hoods['Longitude']
                                  )

print(sf_venues.shape)
sf_venues.head()

Anza Vista
Alamo Square
Sutro Heights
Seacliff
Lake Street
Presidio
Presidio Terrace
Outer Richmond
Outer Sunset
Lakeshore
Inner Richmond
Inner Sunset
Fishermans Wharf
Aquatic Park / Ft. Mason
Union Street
Presidio Heights
Laurel Heights / Jordan Park
Northern Waterfront
Lone Mountain
Panhandle
Haight Ashbury
Ashbury Heights
Castro
Noe Valley
Glen Park
Outer Mission
Marina
Cow Hollow
Pacific Heights
Lower Pacific Heights
Japantown
Polk Gulch
Rincon Hill
Western Addition
Lower Haight
Duboce Triangle
Mint Hill
Mission Dolores
Hayes Valley
North Beach
Russian Hill
Chinatown
Nob Hill
Lower Nob Hill
Cathedral Hill
Downtown
Tenderloin
Civic Center
South of Market
Mission
Bernal Heights
Telegraph Hill
Financial District
South Beach
Showplace Square
Protrero Hill
Mission Bay
Dogpatch
Bayview
Central Waterfront
Golden Gate Heights
Buena Vista
Corona Heights
Cole Valley
Parnassus Heights
Eureka Valley
Dolores Heights
Forest Knolls
Clarendon Heights
Upper Market
Midtown Terrace
Laguna Honda
Fores

Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0   Anza Vista                37.7808               -122.4432   
1   Anza Vista                37.7808               -122.4432   
2   Anza Vista                37.7808               -122.4432   
3   Anza Vista                37.7808               -122.4432   
4   Anza Vista                37.7808               -122.4432   

                               Venue  Venue Latitude  Venue Longitude  \
0                         Anza Vista       37.779721      -122.442065   
1  European Wax Center San Francisco       37.781791      -122.445398   
2                European Wax Center       37.781838      -122.445449   
3        Tony's Cable Car Restaurant       37.782814      -122.444836   
4                             Subway       37.781817      -122.445434   

            Venue Category  
0             Neighborhood  
1  Health & Beauty Service  
2  Health & Beauty Service  
3             Burger Joint  
4           Sandwich Place

In [12]:
nyc_venues = getNearbyVenues(names=nyc_hoods['Neighborhood'],
                                   latitudes=nyc_hoods['Latitude'],
                                   longitudes=nyc_hoods['Longitude']
                                  )

print(nyc_venues.shape)
nyc_venues.head()

Marble Hill
Inwood
Washington Heights
Hamilton Heights
Manhattanville
Morningside Heights
Harlem
East Harlem
Upper West Side
Carnegie Hill
Upper East Side
Roosevelt Island
Columbus Circle
Midtown
Clinton
Sutton Place
Turtle Bay
Tudor City
Garment District
Murry Hill
Chelsea
Flatiron District
Gramercy
Stuyvesant Town
West Village
Greenwich Village
East Village
SoHo
NoHo
Little Italy
Lower East Side
Tribeca
Battery Park
Financial District
Chinatown
(1371, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Marble Hill                40.8758                -73.9105   
1  Marble Hill                40.8758                -73.9105   
2  Marble Hill                40.8758                -73.9105   
3  Marble Hill                40.8758                -73.9105   
4  Marble Hill                40.8758                -73.9105   

               Venue  Venue Latitude  Venue Longitude    Venue Category  
0           Arturo's       40.874412       -73.910271       Pizza Place  
1          Starbucks       40.873755       -73.908613       Coffee Shop  
2           Rite Aid       40.875467       -73.908906          Pharmacy  
3  Subway Sandwiches       40.874667       -73.909586    Sandwich Place  
4           GameStop       40.874267       -73.909342  Video Game Store

### Group the venues by neighborhood

In [11]:
sf_venues.groupby('Neighborhood').count()

Neighborhood Latitude  Neighborhood Longitude  \
Neighborhood                                                              
Alamo Square                                 17                      17   
Anza Vista                                    5                       5   
Apparel City                                 12                      12   
Aquatic Park / Ft. Mason                     29                      29   
Ashbury Heights                               9                       9   
...                                         ...                     ...   
West Portal                                  41                      41   
Western Addition                             19                      19   
Westwood Highlands                            1                       1   
Westwood Park                                 3                       3   
Yerba Buena Island                            6                       6   

                          Venue  Venue Latitude  Venue Longitude  \
Neighborhood                                                       
Alamo Square                 17              17               17   
Anza Vista                    5               5                5   
Apparel City                 12              12               12   
Aquatic Park / Ft. Mason     29              29               29   
Ashbury Heights               9               9                9   
...                         ...             ...              ...   
West Portal                  41              41               41   
Western Addition             19              19               19   
Westwood Highlands            1               1                1   
Westwood Park                 3               3                3   
Yerba Buena Island            6               6                6   

                          Venue Category  
Neighborhood                              
Alamo Square                          17  
Anza Vista                             5  
Apparel City                          12  
Aquatic Park / Ft. Mason              29  
Ashbury Heights                        9  
...                                  ...  
West Portal                           41  
Western Addition                      19  
Westwood Highlands                     1  
Westwood Park                          3  
Yerba Buena Island                     6  

[113 rows x 6 columns]

In [13]:
nyc_venues.groupby('Neighborhood').count()

Neighborhood Latitude  Neighborhood Longitude  Venue  \
Neighborhood                                                                
Battery Park                            50                      50     50   
Carnegie Hill                           25                      25     25   
Chelsea                                 50                      50     50   
Chinatown                               50                      50     50   
Clinton                                 50                      50     50   
Columbus Circle                         40                      40     40   
East Harlem                             35                      35     35   
East Village                            50                      50     50   
Financial District                      50                      50     50   
Flatiron District                       50                      50     50   
Garment District                        50                      50     50   
Gramercy                                10                      10     10   
Greenwich Village                       44                      44     44   
Hamilton Heights                        48                      48     48   
Harlem                                  21                      21     21   
Inwood                                  35                      35     35   
Little Italy                            50                      50     50   
Lower East Side                         28                      28     28   
Manhattanville                           4                       4      4   
Marble Hill                             13                      13     13   
Midtown                                 50                      50     50   
Morningside Heights                     30                      30     30   
Murry Hill                              50                      50     50   
NoHo                                    50                      50     50   
Roosevelt Island                         9                       9      9   
SoHo                                    50                      50     50   
Stuyvesant Town                          8                       8      8   
Sutton Place                            48                      48     48   
Tribeca                                 50                      50     50   
Tudor City                              23                      23     23   
Turtle Bay                              50                      50     50   
Upper East Side                         50                      50     50   
Upper West Side                         50                      50     50   
Washington Heights                      50                      50     50   
West Village                            50                      50     50   

                     Venue Latitude  Venue Longitude  Venue Category  
Neighborhood                                                          
Battery Park                     50               50              50  
Carnegie Hill                    25               25              25  
Chelsea                          50               50              50  
Chinatown                        50               50              50  
Clinton                          50               50              50  
Columbus Circle                  40               40              40  
East Harlem                      35               35              35  
East Village                     50               50              50  
Financial District               50               50              50  
Flatiron District                50               50              50  
Garment District                 50               50              50  
Gramercy                         10               10              10  
Greenwich Village                44               44              44  
Hamilton Heights                 48               48              48  
Harlem                           21